In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import re

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pdt_df = pd.read_csv('full_products_list.csv', dtype={'ID': int})
batches_df = pd.read_csv('batches_list.csv')
recipe_df = pd.read_csv('recipe_list.csv')

In [3]:
xls = pd.ExcelFile(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\duplicate_products.xlsx')
dp_list = pd.read_excel(xls, 'copy_final_list_duplicates')
unique_pdts_df = pd.read_excel(xls, 'unique final list')

In [4]:
new_dp_list = dp_list.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8',
                           'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
                           'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
                           'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [5]:
new_dp_list = new_dp_list.rename(columns={'Eva': 'PRODUCT ID', 'Unnamed: 6': 'Generic Name', 'Unnamed: 7': 'Brand Name'})

In [6]:
new_dp_list = new_dp_list.dropna(axis='rows')

In [7]:
new_dp_list = new_dp_list.drop([new_dp_list.index[-1]])

In [8]:
new_dp_list['PRODUCT ID'] = pd.to_numeric(new_dp_list['PRODUCT ID'], errors='ignore')

In [9]:
unique_pdts_df = unique_pdts_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'RETAIL PRICE PER SMALLEST UNIT', 
                                      'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id', 
                                      'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 
                                      'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 
                                      'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 
                                      'Unnamed: 25'], axis=1)

In [ ]:
# new_dp_list['PRODUCT ID'] = pd.to_numeric(new_dp_list['PRODUCT ID'], errors='ignore')

In [10]:
pdt_df = pdt_df.rename(columns={'ID': 'PRODUCT ID'})

In [ ]:
new_dp_list.info()

In [ ]:
unique_pdts_df.info()

In [ ]:
new_dp_list.head(20)

In [ ]:
for product in unique_pdts_df['GENERIC NAME']:
    # find matches
    matches = process.extract(product.lower(), new_dp_list['Generic Name'].str.lower(), limit=new_dp_list.shape[0])
#     print(matches)
    for match in matches:
        if match[1] >= 25:
            new_dp_list['NEW ID'] = (str(match[0]))
            new_dp_list['NEW ID'] = unique_pdts_df['ID']            

In [ ]:
new_dp_list.head()

In [ ]:
new_dp_list['PRODUCT ID'] = new_dp_list['PRODUCT ID'].astype(int)

In [ ]:
new_dp_list.info()

In [ ]:
unique_pdts_df[unique_pdts_df['ID'] == 3949]

In [11]:
dp_names = new_dp_list['Generic Name'].unique()
uniq_names = unique_pdts_df['GENERIC NAME'].unique()

In [12]:
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for n in list_names:
        score = fuzz.ratio(name, n)
        if (score > min_score) & (score > max_score):
            max_name = n
            max_score = score
        return (max_name, max_score)

In [13]:
new_dp_list['new_id'] = '' 

In [16]:
for p in unique_pdts_df['GENERIC NAME']:
    match = match_names(p, new_dp_list['Generic Name'], 75)
    if match[1] >= 75:
        print(p)
        new_dp_list['new_id'][p] = new_dp_list['PRODUCT ID']
    

Betadine Mouthwash Gargle


<ipython-input-16-70fbb9666e3f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dp_list['new_id'][p] = new_dp_list['PRODUCT ID']


In [15]:
new_dp_list.head()

,PRODUCT ID,Generic Name,Brand Name,new_id
0,3951,Betadine Mouthwash Gargle,Betadine Mouthwash Gargle,
1,387,Montelukast,DENK AIR JUNIOR 5mg,
2,4933,Montelukast Tabs,Denk-air,
3,4942,Montelukast Tabs,Denk-Air Junior,
4,3146,Diclofenac Tabs,Diclomol SR,
